# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「神戸情報大学院大学」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 令和４年４月１日から施行されている神戸情報大学院大学の学則
- 2025年4月1日に改定された神戸情報大学院大学のデジタルパンフレット
- 上記２文書のPDFファイルをyomitokuによりマークダウン化した物

## 扱うモデル

「Qwen/Qwen3-8B-FP8」を使用します。このモデルは、以下の特徴を持ちます。

- 最近出た凄いって言われてるモデルを使ってみたいって感じです。

### 演習環境の準備

In [3]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [4]:
!pip show transformers
# バージョンが古い場合は
#pip install --upgrade transformers

Name: transformers
Version: 4.51.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [2]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import random
random.seed(0)

In [7]:

# モデル(Qwen3-8B)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "Qwen/Qwen3-8B-FP8"
# 2. トークナイザーの読み込み (モデル名に合わせて自動で変更)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True # Qwenモデルはカスタムコードを含む場合があるのでTrueを推奨
)

# 3. BitsAndBytesConfig (量子化設定)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_compute_dtype: Qwenモデルは通常bfloat16で学習されています。
    # GPUがbfloat16に対応している場合 (NVIDIA Ampere世代以降など) は torch.bfloat16 を推奨。
    # 対応していない場合は torch.float16 を使用します。
    # モデルカードのQuickstartでは torch_dtype="auto" となっていますが、
    # 4bit量子化と組み合わせる際は明示的に指定するのが良いでしょう。
    bnb_4bit_compute_dtype=torch.bfloat16, # または torch.float16
    bnb_4bit_quant_type="nf4", # 一般的に高性能なので維持
    bnb_4bit_use_double_quant=False, # まずはFalseで。VRAMが非常に厳しい場合にTrueを検討
)
# 4. モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", # 維持
    quantization_config=bnb_config, # 4bit量子化設定を適用
    # torch_dtype: 量子化時のベースとなるデータ型。bnb_4bit_compute_dtype と合わせるか、
    # モデルがネイティブにサポートする型 (Qwenの場合はbfloat16が適していることが多い) を指定。
    torch_dtype=torch.bfloat16, # または torch.float16
    trust_remote_code=True, # QwenモデルではTrueが必要なことが多い
    # attn_implementation: FlashAttention2が利用可能ならパフォーマンス向上に寄与する可能性
    # attn_implementation="flash_attention_2", # 必要ならインストールして試す
    # rope_scaling: 長いコンテキスト (YaRN) を使う場合に設定。後述。
)

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available. Running on CPU.")

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
Number of GPUs: 1
Current CUDA device: 0
Device name: NVIDIA GeForce RTX 4080


# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [12]:
def generate_output_qwen3(query, model, tokenizer, enable_thinking=True, max_new_tokens=512):
    """
    Qwen3モデル用に調整されたテキスト生成関数。

    Args:
        query (str): モデルへの質問文。
        model: ロード済みのQwen3モデル。
        tokenizer: Qwen3モデルに対応するトークナイザー。
        enable_thinking (bool): Qwen3のThinking Modeを有効にするか。Trueで有効。
        max_new_tokens (int): 生成する最大トークン数。

    Returns:
        str: モデルからの応答テキスト（Thinking Modeの場合は最終回答部分）。
    """
    messages = [
        {"role": "user", "content": query},
    ]

    # 1. チャットテンプレートの適用 (enable_thinking を追加)
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        enable_thinking=enable_thinking # Qwen3特有の引数
    ).to(model.device)

    # 2. 終了トークンの設定
    # Qwen3では、tokenizer.eos_token_id が主要な終了トークンです。
    # <|eot_id|> は他のモデル(Llama 3など)の特殊トークンであり、Qwen3では通常不要か、
    # 存在しない場合はエラーになる可能性があります。
    # Qwen3の特殊トークンには <|im_end|> や <|endoftext|> がありますが、
    # apply_chat_templateがこれらを適切に扱うため、eos_token_idのみで十分なことが多いです。
    eos_token_id_to_use = tokenizer.eos_token_id
    # 必要に応じて他の終了候補もリストに追加できますが、まずはeos_token_idだけで試すのが無難です。
    # terminators = [tokenizer.eos_token_id]
    # もしtokenizer.eos_token_idがNoneの場合は代替を探す
    if eos_token_id_to_use is None and hasattr(tokenizer, 'eod_id'): # Qwen特有の属性
        eos_token_id_to_use = tokenizer.eod_id


    # 3. サンプリングパラメータの設定
    generation_kwargs = {
        "max_new_tokens": max_new_tokens,
        "eos_token_id": eos_token_id_to_use,
        "pad_token_id": eos_token_id_to_use, # warning: `pad_token_id` is not set... を避ける
    }

    if enable_thinking:
        # Thinking Mode (enable_thinking=True) の場合の推奨値
        # モデルカードではgreedy decodingは非推奨なので do_sample=True
        generation_kwargs.update({
            "do_sample": True,
            "temperature": 0.6,
            "top_p": 0.95,
            "top_k": 20,
            # "min_p": 0, # モデルカードに記載あり。必要に応じて追加
        })
    else:
        # Non-Thinking Mode (enable_thinking=False) の場合の推奨値
        generation_kwargs.update({
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.8,
            "top_k": 20,
            # "min_p": 0, # モデルカードに記載あり。必要に応じて追加
        })
    
    # もし strictly greedy な出力を試したい場合 (モデルカードでは非推奨だが比較用として)
    # generation_kwargs = {
    #     "max_new_tokens": max_new_tokens,
    #     "eos_token_id": eos_token_id_to_use,
    #     "pad_token_id": eos_token_id_to_use,
    #     "do_sample": False, # greedy search
    # }

    # 4. テキスト生成
    outputs = model.generate(
        input_ids,
        **generation_kwargs
    )

    # 5. 応答部分の抽出とデコード
    response_ids = outputs[0][input_ids.shape[-1]:]

    if enable_thinking:
        # Thinking Mode の場合、<think>...</think> タグの後に最終回答がある
        full_response_decoded = tokenizer.decode(response_ids, skip_special_tokens=False) # 特殊トークンを含めてデコード
        
        think_end_tag = "</think>"
        final_answer = ""
        
        think_end_idx = full_response_decoded.rfind(think_end_tag)
        if think_end_idx != -1:
            raw_final_answer = full_response_decoded[think_end_idx + len(think_end_tag):]
            # 特殊トークンを除去して最終回答をクリーンにする
            final_answer = tokenizer.decode(tokenizer.encode(raw_final_answer, add_special_tokens=False), skip_special_tokens=True).strip()
        else:
            # <think> タグが見つからない場合は、全体をデコード (特殊トークン除去)
            final_answer = tokenizer.decode(response_ids, skip_special_tokens=True).strip()
        
        return final_answer
    else:
        # Non-Thinking Mode の場合は、単純にデコード
        return tokenizer.decode(response_ids, skip_special_tokens=True).strip()

### 神戸情報大学院大学の詳細と質問内容について


質問:
神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。
資料: KIC_digiPanp.md
質問意図:
デジタルパンフレットから特定の人物（学長）に関する情報を正確に抽出できるか。
箇条書きや特定のセクション（PROFILE）から複数の情報をピックアップする能力を測る。

質問:
神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている入学金と年間の授業料はそれぞれいくらですか？
資料: kic-gakusoku.md
質問意図:
学則の本文と別表を参照し、特定の費用項目（入学金、授業料）の金額を正確に抽出する能力を測る。
複数の箇所に分散している情報を正しく結びつける能力を測る。

質問:
神戸情報大学院大学において「フルオンライン履修制度」を利用して正規課程を修了した場合、学則に基づき授与される学位の正式名称は何ですか？
資料: KIC_digiPanp.md, kic-gakusoku.md
質問意図:
デジタルパンフレットで特定の制度（フルオンライン履修制度）が正規課程であることを理解する。
学則で学位授与に関する条項（第16条）と、そこで言及される可能性のある学位規程（本資料にはないが、第16条で触れられていることから「情報システム修士（専門職）」がパンフレット側から補完されるか、あるいは学則第5条の専攻名から類推できるか）を参照し、学位名を特定する能力を測る。
両資料にまたがる情報を関連付けて回答を生成できるか。パンフレットP.40に「学位：情報システム修士（専門職）」とあるため、これを学則の学位授与と結びつけられるか。

質問:
神戸情報大学院大学では「100カ国を超える国々からの留学生」を受け入れていることが特徴の一つです。この大学の特徴は、学則に定められる入学資格のどの項目によって具体的に支えられていると考えられますか？関連する項目を複数挙げ、その理由を説明してください。
資料: KIC_digiPanp.md, kic-gakusoku.md
質問意図:
デジタルパンフレットに記載された大学の特徴（国際性）と、学則の具体的な規定（入学資格）を結びつけて解釈する能力を測る。
学則第18条の各号（特に(2)外国での課程修了者、(3)文科大臣指定者、(5)同等以上の学力認定者など）が、パンフレットの記述する多様な留学生受け入れにどう貢献するかを論理的に説明できるか。
単なる情報抽出ではなく、規定の意図や適用範囲を推察する能力を測る。

質問：
神戸情報大学院大学に在籍する高原敏竜特任教授は、国際機関のプロジェクト調整員や企画調査員として複数の国での活動経験があります。これらの国々の一般的な開発課題をいくつか挙げてください。その上で、高原特任教授が専門とする分野が、これらの開発課題の解決にどのように貢献できる可能性があるか、具体例を交えて説明してください。
資料: KIC_digiPanp.md
必要な外部知識: ジブチ、セネガル、ニジェール、ベナンといった国々の一般的な開発課題に関する知識、ICT4Dの具体的な応用例に関する知識。
質問意図:
教員の経歴から活動地域を特定し、その地域の開発課題を外部知識で補う。
教員の専門分野（ICT4D, HCI, UX）が、それらの課題解決にどう応用できるかを具体的に考察する能力を測る。
抽象的な専門分野と具体的な社会課題を結びつける応用力、創造力を評価する



#### 解答データ

---
RAGと同じデータを与えたGemini_2.5_pro_preview 05-06にて作成

---

質問1: 神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。

**資料**: KIC_digiPanp.md

**該当箇所と情報**:
*   **ページ**: P.4 (ファイル内では `03 Kobe Institute of Computing` のページ)
*   **学長の名前**: 炭谷 俊樹
*   **PROFILEセクションより**:
    1.  東京大学理学系研究科修士課程修了(物理学専攻)
    2.  ビジネス·ブレークスルー大学大学院客員教授(経営管理専攻·問題解決思考)
    3.  ラーンネット·グローバルスクール代表

**まとめ**:
神戸情報大学院大学の現学長は**炭谷 俊樹**氏です。主なプロフィールは以下の3点です。
1.  東京大学理学系研究科修士課程修了(物理学専攻)
2.  ビジネス·ブレークスルー大学大学院客員教授(経営管理専攻·問題解決思考)
3.  ラーンネット·グローバルスクール代表

---

質問2: 神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている入学金と年間の授業料はそれぞれいくらですか？

**資料**: kic-gakusoku.md

**該当箇所と情報**:
*   **学則 第30条**:
    > 第30条 入学検定料、入学金及び授業料等は、別表1の通りとする。
*   **別表1 入学金、授業料、入学検定料、その他の費用**:
    > | 区 分          | 金 額         |
    > |----------------|---------------|
    > | 入学金         | 200,000円     |
    > | 授業料(1年当たり) | 1,400,000円   |
    > | 施設·設備費(1年当たり) | 200,000円     |
    > | 入学検定料     | 30,000円      |

**まとめ**:
神戸情報大学院大学に入学する場合の費用は、学則第30条および別表1に基づき、以下の通りです。
*   **入学金**: 200,000円
*   **年間の授業料**: 1,400,000円

---

質問3: 神戸情報大学院大学において「フルオンライン履修制度」を利用して正規課程を修了した場合、学則に基づき授与される学位の正式名称は何ですか？

**資料**: KIC_digiPanp.md, kic-gakusoku.md

**該当箇所と情報**:
*   **KIC_digiPanp.md**:
    *   **ページ P.12 (ファイル内 `11 Kobe Institute of Computing`)**:
        > 「この度神戸情報大学院大学では、正規課程(専門職修士課程)をオンラインにより受講することによって、修了に必要なすべての科目を取得し、修士の学位\[情報システム修士(専門職)\]を取得するフルオンライン履修制度を開始します。」
    *   **ページ P.40 (最終ページ)**:
        > 「●学 位:情報システム修士(専門職)」
*   **kic-gakusoku.md**:
    *   **第5条 (課程·専攻)**:
        > 「本大学院に、次の課程、専攻を置く。
        > 情報技術研究科
        > 専門職学位課程 情報システム専攻」
    *   **第16条 (学位の授与)**:
        > 「本大学院において、課程修了の認定を得た者には、神戸情報大学院大学学位規程の定めるところにより学位を授与する。」

**まとめ**:
神戸情報大学院大学の「フルオンライン履修制度」は正規課程であり、これを修了した場合に授与される学位の正式名称は、デジタルパンフレットの記載および学則第5条の専攻名から「**情報システム修士（専門職）**」です。
学則第16条には「神戸情報大学院大学学位規程の定めるところにより学位を授与する」とありますが、パンフレットに具体的な学位名が明記されています。

---

質問4: 神戸情報大学院大学では「100カ国を超える国々からの留学生」を受け入れていることが特徴の一つです。この大学の特徴は、学則に定められる入学資格のどの項目によって具体的に支えられていると考えられますか？関連する項目を複数挙げ、その理由を説明してください。

**資料**: KIC_digiPanp.md, kic-gakusoku.md

**該当箇所と情報**:
*   **KIC_digiPanp.md**:
    *   **ページ P.2 (ファイル内 `01 Kobe Instutute ofc.omputing`)**:
        > 「100カ国を超える学生の多様性」
        > 「開学以降、中国やベトナム、ネパールなどアジアの学生のみならず、ルワンダ、タンザニア、モザンビークなどアフリカ諸国やアフガニスタン、シリアなど中東地域から多く学生を受け入れており、これまで受け入れた学生の出身国は100カ国を超えます。」
*   **kic-gakusoku.md**:
    *   **第18条 (入学資格)**:
        > 「本大学院に入学できる者は、次の各号の一に該当する者であって、かつ所定の選考に合格した者とする。
        > (1) 大学を卒業した者
        > (2) 外国において学校教育における16年の課程を修了した者
        > (3) 文部科学大臣の指定した者
        > (4) 専修学校の専門課程(修業年限が4年以上であることその他の文部科学大臣が定める基準を満たすものに限る。)で文部科学大臣が別に指定するものを文部科学大臣が定める日以後に修了した者
        > (5) その他、大学を卒業したと同等以上の学力があると本大学院において認められた者」

**まとめ**:
神戸情報大学院大学が「100カ国を超える国々からの留学生」を受け入れているという特徴は、学則第18条に定められる以下の入学資格項目によって具体的に支えられていると考えられます。

1.  **「(2) 外国において学校教育における16年の課程を修了した者」**
    *   **理由**: この項目は、日本国外の大学（またはそれに相当する高等教育機関）で正規の学士課程（通常16年）を修了した者を対象としています。これにより、世界各国の大学卒業者が直接的に本大学院の入学資格を得られるため、多様な国からの留学生受け入れの基盤となります。

2.  **「(3) 文部科学大臣の指定した者」**
    *   **理由**: この項目は、文部科学大臣が個別に指定する条件を満たす者に入学資格を認めるものです。これには、特定の国や地域の教育制度に基づく資格や、国際的な教育プログラムの修了者などが含まれる可能性があり、標準的な16年課程以外の多様な教育背景を持つ留学生の受け入れを可能にします。

3.  **「(5) その他、大学を卒業したと同等以上の学力があると本大学院において認められた者」**
    *   **理由**: この項目は、本大学院が独自の判断で、大学卒業者と同等以上の学力があると認めた者に入学資格を与えるものです。これにより、例えば実務経験が豊富であるが学歴が標準的な基準と異なる場合や、特定の分野で顕著な実績を持つ者など、画一的な基準では測れない多様なバックグラウンドを持つ国内外の志願者に対して門戸を開くことができ、国際的な学生の多様性に貢献します。

これらの項目が組み合わさることで、さまざまな教育制度や経歴を持つ世界中の人々が神戸情報大学院大学で学ぶ機会を得られるようになっています。

はい、承知いたしました。修正された質問に対して、回答に必要な情報を資料から抜き出し、コンパクトに整理します。

---

**質問:**
質問5:神戸情報大学院大学に在籍する高原敏竜特任教授は、国際機関のプロジェクト調整員や企画調査員として複数の国での活動経験があります。これらの国々の一般的な開発課題をいくつか挙げてください。その上で、高原特任教授が専門とする分野が、これらの開発課題の解決にどのように貢献できる可能性があるか、具体例を交えて説明してください。

**資料**: KIC_digiPanp.md

**必要な外部知識**:
*   高原特任教授の活動経験国（ジブチ、セネガル、ニジェール、ベナン）の一般的な開発課題
*   ICT4D、HCI、UXの具体的な応用例

**回答に必要な情報と考察のポイント（コンパクト版）:**

1.  **高原特任教授の活動経験国（パンフレットP.14より）:**
    *   ジブチ、セネガル、ニジェール、ベナン

2.  **上記国々の一般的な開発課題（外部知識より例示）:**
    *   **教育**: 教育機会の地域間格差、教育の質の低さ、識字率の課題
    *   **保健医療**: 医療アクセス不良、乳幼児死亡率、感染症対策
    *   **農業・食料安全保障**: 低い農業生産性、気候変動への脆弱性、市場アクセスの困難
    *   **インフラ**: 電気・水道・通信網の未整備、交通網の不備
    *   **経済・雇用**: 貧困、若年層の失業、産業育成の遅れ
    *   **ガバナンス**: 行政サービスの非効率性、情報公開の不足

3.  **高原特任教授の専門分野（パンフレットP.14、P.25より）:**
    *   ICT4D (Information and Communication Technology for Development)
    *   HCI (Human Computer Interaction)
    *   UX (User Experience)
    *   （研究室理念より補足：多様性、文化親和性、人文分野への関心、映像的アプローチ）

4.  **専門分野による開発課題解決への貢献可能性（具体例）：**
    *   **教育課題への貢献例:**
        *   **ICT4D**: 現地の言語や文化に合わせたインタラクティブな教育コンテンツ（HCI/UXデザイン）をモバイル端末で提供し、遠隔地の子供たちや成人非識字者向けの学習機会を創出。
        *   **映像的アプローチ**: 理解しやすい映像教材の開発・普及。
    *   **保健医療課題への貢献例:**
        *   **ICT4D**: 地域保健員向けの診断支援アプリや患者情報管理システム（使いやすいHCI/UX）を開発し、医療サービスの質と効率を向上。妊産婦ケアや予防接種の啓発にSMSや音声メッセージを活用。
    *   **農業課題への貢献例:**
        *   **ICT4D**: 農家向けに気象情報、市場価格、栽培技術情報を提供するモバイルサービス（HCI/UXを考慮したインターフェース）を開発。サプライチェーン管理の透明化。
        *   **UX**: 現地農家のニーズや利用環境を徹底的に調査し、本当に役立つ情報提供の形をデザイン。
    *   **ガバナンス課題への貢献例:**
        *   **ICT4D**: 行政手続きのオンライン化や市民からのフィードバック収集プラットフォーム（アクセシブルなHCI/UX）を構築し、行政の透明性と効率性を向上。
        *   **HCI/UX**: 多様な識字レベルやICTスキルを持つ住民が容易に利用できるデザインを追求。

**コンパクトなまとめ:**
高原特任教授は、ジブチ、セネガル、ニジェール、ベナンでの活動経験があります。これらの国々では、教育格差、医療アクセス不良、低い農業生産性、インフラ未整備といった開発課題が一般的です。
教授の専門であるICT4D、HCI、UXは、これらの課題に対し、
*   **教育**: 現地文化に適応したモバイル学習コンテンツの提供。
*   **保健医療**: 地域保健員向け診断支援アプリや患者管理システムの開発。
*   **農業**: 農家向け気象・市場情報提供モバイルサービスの構築。
*   **ガバナンス**: アクセシブルな行政サービスプラットフォームの構築。
といった形で、現地のニーズと文化を尊重（HCI/UXの視点）した技術活用を通じて貢献できる可能性があります。

### ベースライン

In [13]:
question =  "神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。"
# Thinking Mode を有効にして生成
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")

# Thinking Mode を無効にして生成
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking ON: 神戸情報大学院大学（神戸情報大学院大学）の現学長は**堀之内隆宏氏**です。以下に主なプロフィールを3点挙げます。

1. **学術的背景**  
   堀之内氏は、京都大学大学院理学研究科（情報
Thinking OFF: 神戸情報大学院大学の現学長は**大塚 明（おおつこ あきら）氏**です。以下に主なプロフィールと経歴を3点挙げます。

1. **学術的背景**  
   大塚学長は、京都大学大学院理学研究科応用数学専攻博士課程を修了し、数理科学の分野で幅広い知識と研究経験を持っています。専門は情報科学、特に計算機科学および数理モデルの構築にあります。

2. **大学での役職経験**  
   神戸情報大学院大学では、学長を務める以前に、学長補佐や学部長などの要職を歴任しており、大学運営や教育・研究の推進に深く関わってきました。

3. **社会的活動と役職**  
   大塚学長は、日本学術振興会（JSPS）や日本情報処理学会などの学術団体で活躍し、学術的・教育的リーダーとしての役割を果たしています。また、大学教育の質の向上や産学連携の推進にも積極的です。

これらの経歴から、大塚学長は学術的知見と教育・研究の実践的な経験を兼ね備えたリーダーとして知られています。


In [14]:
question =  "神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている入学金と年間の授業料はそれぞれいくらですか？"
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking ON: <think>
Okay, the user is asking about the enrollment fee and annual tuition fees for Kobe Institute of Computing University, specifically referencing Article 30 of the regulations and the attached table 1. First, I need to confirm the exact name of the university. The user wrote "神戸情報大学院大学" which is "Kobe Institute of Computing University" in English. However, I should check if this is the correct official name. Sometimes, the translation might vary, so maybe it's better to confirm the official English name.

Next, I need to find the current academic regulations (学則) for this university. Since I can't access real-time data or the internet, I'll have to rely on my existing knowledge up to 2023. I remember that many Japanese universities have their fees listed on their official websites, but without direct access, I might need to provide a general structure or suggest where to find the information.

The user is asking specifically about Article 30 and Table 1. In Japanese u

In [16]:
question =  "神戸情報大学院大学において「フルオンライン履修制度」を利用して正規課程を修了した場合、学則に基づき授与される学位の正式名称は何ですか？"
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking ON: <think>
Okay, the user is asking about the official name of the degree awarded to someone who completes the regular course at Kobe Institute of Computing University using the "Full Online Course" system. Let me start by recalling what I know about Japanese universities and their degree naming conventions.

First, I need to confirm if Kobe Institute of Computing University actually has a "Full Online Course" system. I remember that some Japanese universities offer online programs, especially for part-time or distance learning. However, the official name of the degree might differ based on the program structure.

In Japan, degrees are typically called "Bachelor's Degree" (学士), "Master's Degree" (修士), or "Doctorate" (博士). The official name might include terms like "Distance Learning" or "Online" in the title. For example, some universities use "Distance Learning Bachelor's Degree" or "Online Master's Degree."

But I should check if Kobe Institute of Computing University speci

In [17]:
question = "神戸情報大学院大学では「100カ国を超える国々からの留学生」を受け入れていることが特徴の一つです。この大学の特徴は、学則に定められる入学資格のどの項目によって具体的に支えられていると考えられますか？関連する項目を複数挙げ、その理由を説明してください。"
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking ON: <think>
Okay, so I need to figure out which specific provisions in the regulations of Kobe Institute of Computing University support the fact that they accept students from over 100 countries. The question asks for multiple related items and explanations. Let me start by recalling what I know about university regulations and international student admissions.

First, I remember that universities often have specific criteria for international students. These might include language proficiency requirements, academic qualifications, and maybe some specific policies for foreign applicants. Since the university accepts students from many countries, there might be provisions that allow for flexible admission criteria or that accommodate different educational systems.

I think one possible item could be the language requirement. For example, if the university requires English proficiency tests like TOEFL or IELTS, but maybe they have different thresholds for international students

In [18]:
question =  "神戸情報大学院大学に在籍する高原敏竜特任教授は、国際機関のプロジェクト調整員や企画調査員として複数の国での活動経験があります。これらの国々の一般的な開発課題をいくつか挙げてください。その上で、高原特任教授が専門とする分野が、これらの開発課題の解決にどのように貢献できる可能性があるか、具体例を交えて説明してください。"
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking ON: <think>
Okay, I need to answer this question about Professor Toshirou Takahashi at Kobe Institute of Computing. Let me start by understanding the question. The user wants me to list some common development challenges in the countries where he has worked, and then explain how his expertise contributes to solving these issues with specific examples.

First, I should figure out which countries he's worked in. The question mentions "複数の国" (multiple countries), but it doesn't specify which ones. Maybe I should assume some common developing countries where international institutions often work, like Southeast Asia, Africa, or Latin America. Let me think of a few examples: maybe Indonesia, Vietnam, Kenya, or Mexico. These are regions where development challenges are prevalent.

Next, the common development challenges. These could include things like economic inequality, infrastructure deficits, environmental issues, education gaps, healthcare access, and technological gaps. For e

- 数値的な評価も見てみます。RagasにはAnswer Accuracyという評価指標があります。今回はこちらを参考に実装した評価関数を利用して測っていきます。

- 今回はgemmaでは性能が不安定だったので、OpenAIのgpt-4oで評価していきます。従って、scoreの実行はopenAI APIキーを所持している関心がある方のみで良いです。

## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「Inference Time Scaling」について誤った知識を提示しました：
* モデルは従来の「推論時間の短縮」という文脈でInference Time Scalingを解釈しており、これはLLM分野における最新の「Inference Time Scaling」概念（推論時計算資源の最適配分）とは異なる説明になります。

---

# 2. 文字起こしデータの活用

## 2.1 パンフレット及び学則をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: 公式パンフレット及び学則
* **目的**: モデルに「神戸情報大学院大学」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: yomitokuによりPDFからmd化したドキュメントをそのまま与える
* **分割方法**: 学則は章ごと、パンフレットはページごとで区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [4]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


KeyboardInterrupt: 

In [5]:
# チャンク化のために LangChain の Text Splitters をインストール
!pip install langchain-text-splitters

In [1]:
import re
import os
# langchain_text_splitters はチャンク化の際に使うので、ここではインポートは不要
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- 前処理関数の定義 (前回の回答と同様) ---

def preprocess_digipanp_text(text):
    """
    digiPanpファイルのテキストを整形します。
    - <br> タグを改行に置換
    - 画像タグを除去
    - 単独の '0' を除去
    - 過剰な空白や改行を整理
    """
    cleaned_text = re.sub(r'<br\s*/?>', '\n', text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'<img\s+[^>]*>', '', cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'\n\n0\n\n', '\n\n', cleaned_text) # 連続改行に囲まれた単独0
    cleaned_text = re.sub(r'\n\s*\n', '\n\n', cleaned_text) # 過剰な空白行を1つにまとめる
    cleaned_text = cleaned_text.strip()
    return cleaned_text

# --- 分割関数の定義 (前回の回答と同様) ---
# ここでは分割はせず、整形後のテキストをファイル保存するだけなので、これらの関数は直接は使いませんが、定義は必要かもしれません。
# split_by_chapter は学則用
def split_by_chapter(text):
    split_pattern = r"(?m)^\s*(?=第\d+章)"
    chapters_raw = re.split(split_pattern, text)
    documents = [chapter.strip() for chapter in chapters_raw if chapter.strip()]
    return documents

# split_by_digipanp_page は digiPanp用
def split_by_digipanp_page(text):
    page_header_pattern = r"^\s*(?:\d{2}\s+Kobe Instutute ofc\.omputing|Kobe Institute of Computing\s+\d{2}).*$"
    split_pattern = r"(?m)(?=" + page_header_pattern + r")"
    pages_raw = re.split(split_pattern, text)
    documents = [page.strip() for page in pages_raw if page.strip()]
    return documents


# --- ファイルパスの設定 ---
gakusoku_file_path = "AI-E-03/RAG/kic-gakusoku.md"
digipanp_file_path = "AI-E-03/RAG/KIC_digiPanp.md"

# 整形後のファイルを保存するパス
cleaned_gakusoku_path = "AI-E-03/RAG/kic-gakusoku_cleaned.md"
cleaned_digipanp_path = "AI-E-03/RAG/KIC_digiPanp_cleaned.md"


# --- 学則ファイルを処理・整形・保存 ---
if os.path.exists(gakusoku_file_path):
    print(f"'{gakusoku_file_path}' を読み込み中...")
    with open(gakusoku_file_path, "r", encoding="utf-8") as f:
        raw_writedown_gakusoku = f.read()

    # 学則ファイルの前処理 (digipanpほどではないが、改行タグや余分な空白の整理など)
    # 必要に応じて preprocess_digipanp_text とは別の専用関数を作るか、共通関数を調整
    # ここでは preprocess_digipanp_text を流用するが、画像タグや0の除去は学則には不要かもしれない
    print("学則ファイルに前処理を適用中...")
    # 学則には画像や単独0が少なそうなので、シンプルなクリーニングに留める
    cleaned_gakusoku_text = re.sub(r'<br\s*/?>', '\n', raw_writedown_gakusoku, flags=re.IGNORECASE)
    cleaned_gakusoku_text = re.sub(r'\n\s*\n', '\n\n', cleaned_gakusoku_text) # 過剰な空白行を1つにまとめる
    cleaned_gakusoku_text = cleaned_gakusoku_text.strip()

    # 整形後のファイルを保存
    print(f"整形後の学則ファイルを '{cleaned_gakusoku_path}' に保存中...")
    with open(cleaned_gakusoku_path, "w", encoding="utf-8") as f:
        f.write(cleaned_gakusoku_text)
    print("保存完了。")

else:
    print(f"エラー: '{gakusoku_file_path}' が見つかりません。パスを確認してください。")


# --- digiPanpファイルを処理・整形・保存 ---
if os.path.exists(digipanp_file_path):
    print(f"\n'{digipanp_file_path}' を読み込み中...")
    with open(digipanp_file_path, "r", encoding="utf-8") as f:
        raw_writedown_digipanp = f.read()

    # digiPanpファイルに前処理を適用
    print("digiPanpファイルに前処理を適用中...")
    preprocessed_digipanp_text = preprocess_digipanp_text(raw_writedown_digipanp) # digipanp向けの前処理関数を使用

    # 分割後クリーニングで使っていたフッター除去を、ファイル全体の前処理として適用する
    # これにより、ファイル全体からフッター行が削除される
    page_footer_pattern = r"(?m)^\s*(?:\d{2}\s+Kobe Instutute ofc\.omputing|Kobe Institute of Computing\s+\d{2}).*?$"
    cleaned_digipanp_text = re.sub(page_footer_pattern, '', preprocessed_digipanp_text, flags=re.IGNORECASE|re.MULTILINE)
    cleaned_digipanp_text = cleaned_digipanp_text.strip() # 再度トリム

    print("前処理完了。")

    # 整形後のファイルを保存
    print(f"整形後のdigiPanpファイルを '{cleaned_digipanp_path}' に保存中...")
    with open(cleaned_digipanp_path, "w", encoding="utf-8") as f:
        f.write(cleaned_digipanp_text)
    print("保存完了。")

else:
    print(f"\nエラー: '{digipanp_file_path}' が見つかりません。パスを確認してください。")

print("\n--- 整形完了 ---")

# 次のステップは、これらの整形済みファイルを読み込んで、
# RecursiveCharacterTextSplitter でチャンク化することです。
# その際には、split_by_chapter や split_by_digipanp_page のロジックで使った
# 章やページのパターンを separators として利用します。
# (整形済みファイルではこれらのパターン自体は残っている必要があります)

'AI-E-03/RAG/kic-gakusoku.md' を読み込み中...
学則ファイルに前処理を適用中...
整形後の学則ファイルを 'AI-E-03/RAG/kic-gakusoku_cleaned.md' に保存中...
保存完了。

'AI-E-03/RAG/KIC_digiPanp.md' を読み込み中...
digiPanpファイルに前処理を適用中...
前処理完了。
整形後のdigiPanpファイルを 'AI-E-03/RAG/KIC_digiPanp_cleaned.md' に保存中...
保存完了。

--- 整形完了 ---


In [2]:


# --- ファイルパスの設定 ---
gakusoku_file_path = "AI-E-03/RAG/kic-gakusoku_cleaned.md"
digipanp_file_path = "AI-E-03/RAG/KIC_digiPanp_cleaned.md"

# 最終的なドキュメントリストを格納する変数 (前処理後の章/ページ単位)
all_documents_raw_chunks = []

# --- 学則ファイルを処理・分割 ---
if os.path.exists(gakusoku_file_path):
    print(f"'{gakusoku_file_path}' を読み込み中...")
    with open(gakusoku_file_path, "r", encoding="utf-8") as f:
        cleaned_gakusoku_text = f.read()

    # 学則ファイルを章単位で分割
    print("学則ファイルを章単位で分割中...")
    chapters = split_by_chapter(cleaned_gakusoku_text)
    all_documents_raw_chunks.extend(chapters)  # 章をリストに追加

else:
    print(f"エラー: '{gakusoku_file_path}' が見つかりません。パスを確認してください。")
# --- digiPanpファイルを処理・分割 ---  
if os.path.exists(digipanp_file_path):
    print(f"\n'{digipanp_file_path}' を読み込み中...")
    with open(digipanp_file_path, "r", encoding="utf-8") as f:
        cleaned_digipanp_text = f.read()

    # digiPanpファイルをページ単位で分割
    print("digiPanpファイルをページ単位で分割中...")
    pages = split_by_digipanp_page(cleaned_digipanp_text)
    all_documents_raw_chunks.extend(pages)
else:
    print(f"\nエラー: '{digipanp_file_path}' が見つかりません。パスを確認してください。") 

print("\n--- 分割完了 ---")
# ここまでで、all_documents_raw_chunks に学則と digiPanp の章/ページ単位のテキストが格納されています。
# これを SentenceTransformer を使ってエンコードし、ベクトルデータを生成します。

'AI-E-03/RAG/kic-gakusoku_cleaned.md' を読み込み中...
学則ファイルを章単位で分割中...

'AI-E-03/RAG/KIC_digiPanp_cleaned.md' を読み込み中...
digiPanpファイルをページ単位で分割中...

--- 分割完了 ---


In [3]:
#エンベディング
print("\n統合ドキュメントの埋め込みを計算中...")

batch_size_to_try = 12
document_embeddings = emb_model.encode(all_documents_raw_chunks, batch_size=batch_size_to_try, show_progress_bar=True)
print("埋め込み計算完了。")


統合ドキュメントの埋め込みを計算中...


NameError: name 'emb_model' is not defined

### 質問１

In [8]:
# Retrievalの実行
question =  "神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。"

query_embeddings = emb_model.encode([question], prompt_name="query")

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[76.52867889404297, 69.59257507324219, 66.40689086914062, 70.59712219238281, 63.67460632324219, 62.597991943359375, 71.0243148803711, 65.73529052734375, 66.55303192138672, 65.32673645019531, 62.35420227050781, 64.91595458984375, 65.5946044921875, 68.03087615966797, 62.89073944091797, 67.8004150390625, 65.78376770019531, 63.62736129760742, 75.80047607421875, 75.08966064453125, 70.82876586914062, 75.12551879882812, 71.06842041015625, 68.59691619873047, 79.2448501586914, 77.6028060913086, 75.81195068359375, 70.83084869384766, 68.8427963256836, 76.62924194335938, 71.97296905517578, 72.14350891113281, 65.48713684082031, 66.69076538085938, 67.80269622802734, 68.67236328125, 67.12065124511719, 68.15283203125, 67.66827392578125, 66.76039123535156, 66.18714141845703, 64.64370727539062, 63.25019454956055, 65.47422790527344, 65.14865112304688, 64.74373626708984, 63.35101318359375, 62.428611755371094, 65.4915771484375, 64.3478775024414, 63.75529861450195, 63.365386962890625, 62.566768646240234, 6

In [13]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(chunked_documents[index], "\n\n")

取得したドキュメント1: (Score: 79.2448501586914)
# 探究型人材へとグレードアップすれば、<br>あなたの活躍フィールドは無限に

神戸情報大学院大学<br>学長代理<br>福岡 賢二

# PROFILE

博士\(学術\)専門分野:Computer & Communication \(コンピュータと情報コミュニケーション\)<br>スウィフト·エックスアイ株式会社 代表取締役社長<br>Tankyu X 代表取締役社長<br>KTGM合同会社 代表<br>IEEE\(米国電気電子学会\)Education Society会員 


取得したドキュメント2: (Score: 78.05462646484375)
# プロフィール

【現職】<br>】 神戸情報大学院大学特任教授/a2ee株式会社 取締役CTO/株式会社quality design 研究所 代表取締役

【経歴】 名古屋大学大学院環境学研究科都市環境工学専攻 博士課程修了 博士\(環境学\)<br>日建設計、日建設計シビル取締役、JR東海コンサルタンツ設計アドバイザー<br>神戸大学客員教授、大阪大学客員教授を経て現職

# 学生へのメッセージ 


取得したドキュメント3: (Score: 77.6028060913086)
神戸情報大学院大学の母体である神戸電子専門学校は昭和33年に<br>設立されました。半世紀以上をかけて約2万人を産業界に輩出して<br>きたICT人材育成のパイオニアです。1995年阪神・淡路大震災の際<br>には、同校研究所が被災地の情報をパソコン通信やインターネットを<br>活用して整理·配信する情報拠点となり、当時大変話題となりました。<br>この時生まれた本学の「Social Innovation by ICT & Yourself」<br>\(ICTと人間力による社会課題解決\)というスピリッツは、今でも多く<br>の若者に影響を与えています。

近年、アフリカ等開発途上国が共通に抱える「貧富の差」解消のため<br>に、富が一部に集中しがちな資源ビジネスに頼らない産業開発、つま<br>り「Socio Economy」への転換が期待されています。そして比較的 


取得したドキュメント4: (Score: 76.62924194335938

In [14]:
references = "\n".join(["* " + chunked_documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。"
response_off = generate_output_qwen3(query, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF with RAG: {response_off}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Thinking OFF with RAG: **質問：神戸情報大学院大学の現学長の名前と主なプロフィール（経歴や役職）を3点挙げてください。**

**回答：**

1. **名前：福岡 賢二**  
   - 現在、神戸情報大学院大学の学長代理を務めています。

2. **専門分野：コンピュータと情報コミュニケーション（Computer & Communication）**  
   - 博士（学術）の学位を持ち、コンピュータおよび情報通信分野における専門知識を有しています。

3. **主な役職と経歴：**  
   - スウィフト·エックスアイ株式会社 代表取締役社長  
   - Tankyu X 代表取締役社長  
   - KTGM合同会社 代表  
   - IEEE（米国電気電子学会）Education Society会員  
   - 神戸情報大学院大学 特任教授  
   - 神戸電子専門学校（本学の母体）出身  

以上が、神戸情報大学院大学の現学長代理である福岡 賢二氏の主なプロフィールです。


In [34]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

0.0


### 質問2

In [17]:
# Retrievalの実行
question =  "神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている入学金と年間の授業料はそれぞれいくらですか？"

query_embeddings = emb_model.encode([question], prompt_name="query")

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[79.25608825683594, 73.24353790283203, 70.76908874511719, 74.50679779052734, 67.30567169189453, 67.331787109375, 73.62245178222656, 71.65143585205078, 71.75489807128906, 69.52690124511719, 67.73352813720703, 77.76113891601562, 73.47357177734375, 67.91435241699219, 64.55523681640625, 72.97860717773438, 79.1497573852539, 72.58626556396484, 71.9549789428711, 73.02268981933594, 65.87992858886719, 66.69898223876953, 65.8152084350586, 67.30955505371094, 68.71100616455078, 73.21908569335938, 73.57707214355469, 68.3267822265625, 66.1386947631836, 70.01744842529297, 67.45393371582031, 69.36210632324219, 62.924861907958984, 63.7194709777832, 64.51844787597656, 65.85379028320312, 63.60061264038086, 61.733890533447266, 62.85055923461914, 69.2947998046875, 65.4560775756836, 63.77598190307617, 63.34384536743164, 63.35735321044922, 63.64483642578125, 63.410587310791016, 60.3902473449707, 60.521305084228516, 64.6829833984375, 64.07672882080078, 63.38351821899414, 61.47804260253906, 63.373573303222656

In [18]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(chunked_documents[index], "\n\n")

取得したドキュメント1: (Score: 79.25608825683594)
# 第1章 総 則

# \(目 的\)

# 神戸情報大学院大学学則

第1条<br>神戸情報大学院大学\(以下「本大学院」という。\)は、人間力を有する高度 ICT<br>人材の育成を目的とする。

\(自己評価\)

第2条<br>2<br>本大学院は、前条の目的を達成するため、教育研究活動の点検及び評価を行う。<br>前項の点検及び評価の組織及び方法は、これを別に定める。

\(課 程\)

第3条<br>本大学院に設置する課程は、専門職学位課程とする。

# 第2章 研究科、修業年限及び収容定員

# \(研究科\)

第4条<br>本大学院に、情報技術研究科を置く。

\(課程·専攻\)

第5条<br>本大学院に、次の課程、専攻を置く。

情報技術研究科

専門職学位課程 情報システム専攻

\(修業年限及び在学年限\)

第6条<br>本大学院の修業年限は、2年を標準とする。 


取得したドキュメント2: (Score: 79.1497573852539)
|平成28年度||
|-|-|
|入学定員|収容定員|
|55名|85名|

11<br>この学則は、令和2年6月1日から施行する。ただし別表 1の授業料の適用は令和3<br>年度新入学生以後の者を対象とする。

12<br>この学則は、令和4年4月1日から施行する。

# 別表1 入学金、授業料、入学検定料、その他の費用

|区 分|金 額|
|-|-|
|入学金|200,000円|
|授業料\(1年当たり\)|1,400,000円|
|施設·設備費\(1年当たり\)|200,000円|
|入学検定料|30,000円|

# 別表2 在籍料

|区 分|金 額|
|-|-|
|在籍料\(1期当たり\)|20,000円|

# 別表3 科目等履修生の授業料、その他の費用

|区 分|金 額|
|-|-|
|登録料|10,000円|
|授業料\(1単位当たり\)|40,000円|
|実験科目加算額\(1単位当たり\)|40,000円|

# 別表 4 研究生の研究指導料、選考料

|区 分|金 額|
|-|-|
|研究指導料\(1期当たり\)|80,000円|

6 


取得したドキュメント3: (Score: 77

In [19]:
references = "\n".join(["* " + chunked_documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている入学金と年間の授業料はそれぞれいくらですか？"
response_off = generate_output_qwen3(query, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF with RAG: {response_off}")

Thinking OFF with RAG: 神戸情報大学院大学に入学する場合、学則第30条および別表1に定められている**入学金**と**年間の授業料**は以下の通りです。

- **入学金**：200,000円  
- **年間の授業料**：1,400,000円  

これらは、学則第30条および別表1に記載されている内容に基づいています。


### 結果 (初期RAG実装)

講義内容のドキュメントを追加したにもかかわらず、モデルの回答には依然として以下の問題が見られます：
* 「高速に推論する」など、従来の一般的な推論最適化と「Inference Time Scaling」を混同した誤った解釈が継続
* 講義内容を参照しているものの、概念の本質を正確に捉えられていない

### 問題分析
以下の要因が考えられます：
1. **ドキュメント品質の問題**: 音声認識による文字起こしの精度不足
2. **検索精度の課題**: 単純な文単位の分割では文脈が失われ、関連性の高いドキュメント片を適切に取得できていない可能性

## 2.2 書き起こしテキストの品質改善

日本語の音声認識（speech2text）モデルは精度に課題があることが知られています。以下に「LLMにおけるInference Time Scalingとは？」に関連する講義内容の書き起こしテキストを比較します：

### 講義中の該当発言 (LLM講座Day4後半から抜粋)
---

<修正前>
---

講義に戻ります。ちょっと練習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドって話で**生のGENIACLM**の話をして終わろうと思いですねちょっとモチベーションから話すと、ちょっと頭で考えてみてほしいとか見れば一瞬で思うとんですけどバナナの色は何ですかって言われたときと、今日の講義聞いた上で、**ゲームソフトの問題は何だと思いますか**って聞かれたとき、多分あの考えることが違うと思うんですね。**羽の色なんですか**っていうと一瞬黄色ですねもしかしたら緑かもしれないけどぐらいですかね物によるかなみたいなおもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**スケール足の問題なんだと思いますか**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだからどうだろうこの辺が問題かなみたいな考えとやっぱ思考としては違うってことは何となく思うかなと思います。なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**スケールって言ってる7Gのスケール**って言ってるのはこういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、あの考えなきゃいけない問題に対しては、考える時間を、に計算式を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。二つで、ちょっと順番が前後しますけどこれの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**大湾**と呼ばれる論文が論文じゃないか、モデルが**オペル**から出ましたプレビューとして出てますけどこの法案で注目されていますこれあの**論文にROMってかブログ**にあるとイエスって右側が訓練時の計算資源をスケールさせたときに、初めて何かロジックのベンチマークがあるんですけどこれをがどうなったかで何となくスケールしてると右側がテストTimeコンピュートっていうふうに書いてると思うんすけど、**水温時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだんコース計算式を増やしていったときに、性能がどう変わるかっていうのでこれもスケールしていってるということがわかると思います。こういった形で、要は考える時間をどうやら推論時に使うと計算資源を推論使うのはいいことがありそうだということがわかります。

---
<修正後>
---

講義に戻ります。ちょっと演習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドってことで**「推論時のスケーリング」**についての話をして終わろうと思います。モチベーションから話すと、ちょっと頭で考えてみてもらえれば一瞬でわかると思うとんですけど、「バナナの色は何ですかって言われたとき」と、今日の講義聞いた上で、**「スケール則の問題は何だと思いますか」**って聞かれたとき、多分あの考えることが違うと思うんですね。
**「バナナの色なんですか」**っていうと黄色ですね。もしかしたら緑かもしれないけど、物によるかなみたいな、おもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**「スケール則の問題なんだと思いますか」**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだから「どうだろう」「この辺が問題かな」みたいな考えとはやっぱ思考としては違うってことは何となく思うかなと思います。
なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**推論時のスケールって言ってるのは**こういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、深く考えなきゃいけない問題に対しては、考える時間に計算資源を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。
これの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**o1**と呼ばれるモデルが**OpenAI**から出ました。プレビューとして出てますけどこのo1で注目されています。これあのo1の**論文ってかブログ**にある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。右側がtest-time computeっていうふうに書いてると思うんすけど、**推論時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います。
こういった形で、要は考える時間をどうやら推論時に使うと、つまり計算資源を推論時に使うのはいいことがありそうだということがわかります。






---
### 文字起こしの誤り

上記の比較からわかるように、音声認識による書き起こしには重大な誤りが多数含まれています：
* 「スケール則の問題」→「ゲームソフトの問題」
* 「o1」→「大湾」
といった明らかに文脈に合わない単語変換が発生しています。

`LLM2024_day4_raw.txt`の中には、このような誤変換が多数見られます。これらの誤りはRAG性能に直接影響し、モデルの回答精度を低下させる要因となります。

したがって、**ドキュメント品質の改善**を行い、RAG性能の向上を図ります。

## 再開

## 講義内容をソースとして活用：改善版RAG実装

* **ドキュメント処理**:
  - speech2textによる書き起こしテキストを人手で丁寧に修正
  - 専門用語（Inference Time Scaling、GPT-o1など）の正確な表記を確保
  - 文脈の流れを維持しつつ、文法的に正確な日本語に修正

* **検索手法**:
  - 引き続き「。」（句点）で区切られた文単位でテキストを分割
  - 文単位の検索により、モデルの入力トークン制限内で関連情報を最大化

この改善により、モデルが正確な情報に基づいて「Inference Time Scaling」の概念を理解し、適切な回答を生成することが期待されます。

In [35]:
with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4.txt", "r") as f:
  raw_writedown = f.read()

In [36]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[310])

ドキュメントサイズ:  350
ドキュメントの例: 
 それからBest of Nとはちょっと違う方法として、N個を生成した後に、それらを集約するという意味では、Day2でやったSelf-Consistencyをこの枠組みの一つとして説明されます


In [37]:
# Retrievalの実行
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[61.82585906982422, 66.46412658691406, 60.23835372924805, 51.8336296081543, 54.9107551574707, 51.56048583984375, 58.39980697631836, 60.80604934692383, 55.0720100402832, 59.28540802001953, 58.47719955444336, 62.86042785644531, 58.22374725341797, 59.10657501220703, 55.343997955322266, 57.3587646484375, 59.760581970214844, 51.49699020385742, 60.624359130859375, 62.524269104003906, 57.502044677734375, 58.56466293334961, 59.720706939697266, 59.44036102294922, 63.487945556640625, 61.78038024902344, 59.621238708496094, 62.45943832397461, 53.73781967163086, 61.90666961669922, 55.26149368286133, 56.99132537841797, 62.45279312133789, 56.23671340942383, 56.53040313720703, 56.40115737915039, 58.694862365722656, 51.75257873535156, 56.227569580078125, 56.70087432861328, 58.3699951171875, 55.44434356689453, 59.27926254272461, 59.53657531738281, 57.6950798034668, 54.768287658691406, 58.02783966064453, 57.402347564697266, 56.4503059387207, 52.4367561340332, 61.598182678222656, 62.70309066772461, 57.00

In [38]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 67.38106536865234)
最後に補足して僕のパート終わろうと思いますけど、同じ計算資源のときにパラメータ増やすのよりも推論資源を増やすのが有効なのかっていうのが問いとしてあると思いますけど、o1の場合だと、訓練時のスケールは同じままって推論時のスケールを増やしたら、より賢くなりましたって話でしたけど、どっちにするのがいいのかっていう意味で言うと、GoogleDeepMindが8月に論文としてまして、Scaling LLM Test-Time Comupte Optimally can be more Effective than Scaling More Paremetersっていうことで、良いらしいというふうに言われてます 


取得したドキュメント2: (Score: 66.58795166015625)
右側がtest-time computeっていうふうに書いてると思うんすけど、推論時に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います 


取得したドキュメント3: (Score: 66.46412658691406)
あのスケールするっていうところではタイトルの通りなんですけど、ちょっとこれスケーリングPretraining回ってなってるんですけれども、ちょっと最近はですね、このPretrainingだけではなくて、推論をスケールさせるというような話も出てきてましてせっかくなのでその最近の話題ということです推論時のスケーリングことで、ちょっとタイトル詐欺が入ってるんですけどPretrainingだけじゃない、スケーリングも扱うということで、ちょっと若干あのタイトル詐欺なんですけども、あの最近の話題ということで推論時のスケジュールについても話していきたいなと思っています 


取得したドキュメント4: (Score: 66.20881652832031)
Trasnformerの場合はスケール則が、パラメータ数が横軸になってますけどこういうふうになると、LSTMの場合には1層2層4層みたいにそれぞれスケール則を解くとこんなふうになります

In [39]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
response = generate_output(query)
print(response)

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時に計算資源を増やす**ことを指します。 

**具体的には:**

* **Pretraining**段階でモデルのパラメータ数を増やす
* **推論時**に、計算資源を増やすことで、モデルの性能を向上させる

**GoogleDeepMindの研究**では、**推論時のスケール**を**最適化**することで、**パラメータ数の増やすよりも、推論資源を増やす方が効果的**であることが示唆されています。


**スケールする際の具体的な方法:**

* **Pretraining**: モデルの学習段階で、パラメータ数を増やす
* **推論時**:  計算資源を増やして、モデルの処理能力を向上させる


 





In [40]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (修正テキストによるRAG)

書き起こしテキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* 「推論時に計算資源を増やすこと」という概念を正確に捉えるようになった

### 問題点
* 「Pretraining時」という記述は講義内容と矛盾している

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 単独の文から情報を抽出するため、講師の全体的な主張や議論の流れを把握できない
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される


---



# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [41]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
response = generate_output(query)
print(response)

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時の計算資源を増やすことで、モデルの性能を向上させる**ことを指します。 

具体的には、**モデルの学習時に使用する計算資源と、推論時に使用する計算資源のバランス**を調整することで、より高速で正確な推論を実現します。


**Inference Time Scalingのポイント**

* **スケーリングの目的:**  推論時の計算資源を増やすことで、モデルの性能を向上させる
* **スケーリング方法:**  Pretraining段階でのスケーリングに加え、推論段階でのスケーリングも検討
* **影響:**  スケーリングによって、モデルの精度や処理速度が変化する
 



 





In [42]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇




# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

In [43]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = []
for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:

  query = f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。[参考資料]\n{ref}\n\n[質問] LLMにおけるInference Time Scalingとは？"
  response = generate_output(query)

  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)



対象となるドキュメント:
 これでほぼちょうどですけど、最後に少しあの、前半では全体の訓練時のスケーリングをする話を基本的にしましたけど、最近ではこの推論時の計算量っていうのも注目するような研究が増えてきています。
代表的なGPT-o1とかですごく注目されてるかなと思いますし、今までやった方法、学んだ方法も結構出てきたと思いますけど、Promptingを工夫するとか、Decodingを工夫するとかいうので、それにも発展的な方法がいろいろ出てきていますし、Meta Generationっていうような枠組みで、DecodingだけじゃなくてそのDecodeした結果を最後どう使うかみたいな含めて、Meta Generationというふうに呼んでますけど、Paralell SearchとかStep Level SearchとかRefinementと言われるような枠組みの研究も出てきていますというような話をしました。
最後に補足して僕のパート終わろうと思いますけど、同じ計算資源のときにパラメータ増やすのよりも推論資源を増やすのが有効なのかっていうのが問いとしてあると思いますけど、o1の場合だと、訓練時のスケールは同じままって推論時のスケールを増やしたら、より賢くなりましたって話でしたけど、どっちにするのがいいのかっていう意味で言うと、GoogleDeepMindが8月に論文としてまして、Scaling LLM Test-Time Comupte Optimally can be more Effective than Scaling More Paremetersっていうことで、良いらしいというふうに言われてます。
厳密に言うとこれなんかタスクによって違うということなので、良いとまで言っていいのかちょっと若干誇大広告な気が個人的にはしてますけど、そういったことを検証するような研究も出てきていますので興味ある人は見てもらえばと思います

関連しているかどうか:  yes 





対象となるドキュメント:
 プレビューとして出てますけどこのo1で注目されています。
これあのo1の論文ってかブログにある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。
右

In [44]:
print(len(references))

4


上記より、上位4件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。（生成内容が確立的なので、4件でない可能性もあります）

In [45]:
query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
response = generate_output(query)
print(response)

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時の計算資源を増やす**ことで、**モデルの性能を向上させる**ことを指します。 

具体的には、**訓練時のスケール**と**推論時のスケール**の両方を検討し、**最適なスケール設定**を見つける研究が進んでいます。


**Inference Time Scaling のポイント**

* **スケールする対象**:  従来は**Pretraining**のみを対象としていましたが、近年では**推論時のスケール**も重要視されています。
* **目的**:  **推論時の計算資源を増やす**ことで、**モデルの性能を向上**させることを目指しています。
* **研究**:  **Scaling Law**と呼ばれる研究で、**Pretraining**だけでは**性能に影響がない**という結果も出ています。
* **スケール方法**:  **モデルのDimension、埋め込みの数次元**などを考慮し、**最適なスケール設定**を見つけることが重要です。




 





In [46]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。

# 5. さらなる改善案: 意味的チャンク化

文単位での分割と前後文脈の追加という現在のアプローチをさらに発展させる手法として、**意味的なチャンク化**が考えられます：

* **意味的チャンク（段落）単位での分割**:
  - 単純な文の区切りではなく、意味的なまとまり（トピック、議論、例示など）に基づいてテキストを分割
  - 人間の主観に基づく意味的な段落分けを活用
  - 各チャンクが「一つの完結した考え」を表現するようにする

* **期待される効果**:
  - より自然な文脈理解が可能に（人間の思考や会話の流れに近い）
  - トピックの開始から結論までの流れを維持できる
  - 概念間の関係性や比較が同一チャンク内に含まれ、より深い理解につなげる

* **検証方法**:
  - 人間が主観的に意味でグループ化したチャンクセットを用意
  - 同じRerank手法を適用し、文単位チャンクとの性能差を比較
  - 回答の正確性、一貫性、網羅性を評価指標として使用

この意味的チャンク化手法は、特に講義のような構造化された発話においては、より自然で効果的な情報検索と理解を可能にすると予想されます。

**注意事項**

**ここから先のセルを実行した場合、GPUメモリ不足になる可能性が高いです。**


このノートブックでは、GPUはT4を使用しています。
Colab Pro等を契約し、L4などのよりGPUメモリの大きいものを使用するか、モデルやその設定等を変更するなどの工夫が必要になります。

In [47]:
# 本来は段落をそのままdocumentsに入れずに一定のサイズに分割した方が良いでしょうが、簡単のために段落をそのまま入れてしまいます。
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[30])

ドキュメントサイズ:  45
ドキュメントの例: 
 具体的な求め方についても話します。 さっきからチラチラ言ってた通りなんすけど基本的にこれどう図るかっていうと、基本的にはいくつかの条件で実験してフィッティングするって言ってんのは、すごい単純に言ってしまうとそうなります。左側GPT4の論文から取ってきた図で説明したもんですけど、グレーのやつを例えば実験してみて、これぐらいのロスになるんだなっていうので、フィッティングするとこういうカーブになります。 ちなみにこれ、なんでこれ直線にならないんだっていうのをすぐ説明しなかったですがこれ縦軸が実は普通のロスと違ってBits-per-wordっていうのになってて、多分2乗スケールのロスになってるからだと思います。 右側も同じですね。この各点について何かいろんな設定で実験してやって、それを結果を見るということをしてますけどよくよく考えるとスケールさせるときにモデルサイズどうすればいいんでしたっけとか、何をどういじるとモデルサイズが大きくなるんでしたっけ、どういうふうに言えばいいんでしたっけとかですね。 あのモデルサイズ変えたときにハイパーパラメータってどうすんでしたっけそういった細かい問題が出てくる。最初の方ですけどモデルサイズどう変化させるかっていうので、前回やった、こういう図があると思いますけどモデルサイズ変えようと思ったら別にパラメータ、層の数を増やしても、いいわけですし、この埋め込みの次元各tokenの次元を増やしてもいいわけですし、各随所に出てくるこのフィードフォワードネットワークっていうのの中間層の次元を上げてもいいですしヘッドを増やしてもそういうのあのパラメータ自体は上がるということで、これどれをどのぐらいやるんですかっていうのが細かく考えると重要になってきます。 この辺は元の論文でも一応議論されてまして、これ三つほど出してるんすけど例えば真ん中のがアスペクト比っていう、モデルのエンベディングのサイズですね。dモデルっていうものを層数で割ったもの、アスペクト比という縦横比みたいなもので幅と深さの比率をアスペクト比っていうふうにこの論文では呼んでいますけど。こういったものを変えて実験してみたっていうのが最初の最初じゃないOpenAIのScaling Lawで話されていました。基本的にはこの辺見るとなんかあんまり性

In [49]:
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
# 簡単のためにtop2でやります。結果を見てもらえれば問題なく関連する項目のみ取得できているのが分かるかと思います。
topk = 2
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
reference = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
response = generate_output(query)
print(response)

In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)